In [1]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"
print(python_version)

Mounted at /content/drive
3.11


In [ ]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`

input_dir = '/content/drive/MyDrive/alphafold2/unfiltered.fasta' #@param {type:"string"}
result_dir = '/content/drive/MyDrive/alphafold2/result_NMR_unfiltered' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 1 #@param [1,2,3,4,5] {type:"raw"}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
stop_at_score = 100 #@param {type:"string"}
#@markdown - early stop computing models once score > threshold (avg. plddt for "structures" and ptmscore for "complexes")
use_custom_msa = False
num_relax = 0 #@param [0, 1, 5] {type:"raw"}
use_amber = num_relax > 0
relax_max_iterations = 200 #@param [0,200,2000] {type:"raw"}
use_templates = False #@param {type:"boolean"}
do_not_overwrite_results = True #@param {type:"boolean"}
zip_results = False #@param {type:"boolean"}

In [ ]:
#@title Install dependencies
%%bash -s $use_amber $use_templates $python_version

set -e

USE_AMBER=$1
USE_TEMPLATES=$2
PYTHON_VERSION=$3

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  if [ -n "${TPU_NAME}" ]; then
    pip install -q --no-warn-conflicts -U dm-haiku==0.0.10 jax==0.3.25
  fi
  ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold
  ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold
  touch COLABFOLD_READY
fi

# Download params (~1min)
python -m colabfold.download

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    conda config --set auto_update_conda false
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python="${PYTHON_VERSION}" 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=8.2.0 python="${PYTHON_VERSION}" pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
#@title Run Prediction

import sys

from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path

# For some reason we need that to get pdbfixer to import
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

setup_logging(Path(result_dir).joinpath("log.txt"))

queries, is_complex = get_queries(input_dir)
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    num_relax=num_relax,
    relax_max_iterations=relax_max_iterations,
    msa_mode=msa_mode,
    model_type="auto",
    num_models=num_models,
    num_recycles=num_recycles,
    model_order=[1, 2, 3, 4, 5],
    is_complex=is_complex,
    data_dir=default_data_dir,
    keep_existing_results=do_not_overwrite_results,
    rank_by="auto",
    pair_mode="unpaired+paired",
    stop_at_score=stop_at_score,
    zip_results=zip_results,
    user_agent="colabfold/google-colab-batch",
)

2025-05-18 15:03:42,687 Running on GPU
2025-05-18 15:03:43,069 Found 5 citations for tools or databases
2025-05-18 15:03:43,070 Query 1/15324: 11570_1_2_2 (length 5)


PENDING:   0%|          | 0/150 [elapsed: 00:00 remaining: ?]

2025-05-18 15:03:43,785 Sleeping for 9s. Reason: PENDING


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:11 remaining: 00:00]


2025-05-18 15:04:06,562 Padding length to 15
2025-05-18 15:04:28,428 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=75.8 pTM=0.0778
2025-05-18 15:04:31,137 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=48.4 pTM=0.0709 tol=4.98
2025-05-18 15:04:33,846 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=47.4 pTM=0.0701 tol=11.4
2025-05-18 15:04:36,555 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=19.2 pTM=0.00274 tol=3
2025-05-18 15:04:36,556 alphafold2_ptm_model_1_seed_000 took 30.0s (3 recycles)
2025-05-18 15:04:36,566 reranking models by 'plddt' metric
2025-05-18 15:04:36,567 rank_001_alphafold2_ptm_model_1_seed_000 pLDDT=19.2 pTM=0.00274
2025-05-18 15:04:36,934 Query 2/15324: 2228_1_1_1 (length 5)


...

2025-05-18 18:50:25,700 Sleeping for 9s. Reason: RATELIMIT


COMPLETE: 100%|██████████| 150/150 [elapsed: 01:21 remaining: 00:00]


2025-05-18 18:50:37,690 Padding length to 15
2025-05-18 18:50:40,606 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=71.2 pTM=0.0414
2025-05-18 18:50:43,341 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=67.7 pTM=0.041 tol=0.451
2025-05-18 18:50:46,085 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=71.8 pTM=0.042 tol=0.773
2025-05-18 18:50:48,832 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=71.8 pTM=0.0419 tol=0.294
2025-05-18 18:50:48,833 alphafold2_ptm_model_1_seed_000 took 11.1s (3 recycles)
2025-05-18 18:50:48,852 reranking models by 'plddt' metric
2025-05-18 18:50:48,853 rank_001_alphafold2_ptm_model_1_seed_000 pLDDT=71.8 pTM=0.0419
2025-05-18 18:50:49,198 Query 214/15324: 34637_1_1_1 (length 10)


SUBMIT:   0%|          | 0/150 [elapsed: 00:00 remaining: ?]

2025-05-18 18:50:49,884 Sleeping for 6s. Reason: RATELIMIT


In [ ]:
# find augumentation in the protein seq for more common dataset.